In [79]:
import os
import zipfile
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import pickle

# SQL Data
> - Unzipping file (im.db.zip), making a connection, conn and assigning a cursor
> - Opening file as a dataframe
> - Creating a single table with relevant data for us to use:

In [80]:
# Check if 'im.db' exists, unzip if it doesn't
if not os.path.exists('zippedData/im.db'):
    with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
        zip_ref.extractall('zippedData')

# Connect to the SQLite database
conn = sqlite3.connect('zippedData/im.db')

In [81]:
cursor = conn.cursor()

In [82]:
pd.read_sql("""
  SELECT *
  FROM sqlite_master  
  """, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


# SQL Data Cleaning and Joining
> > - Movie_ID, Genres, Avg_Rating, Num_Votes, Region, Category, Artist_Name, Screen_Time, and Movie_Title
> > - Where region is United States with Number of votes greater than 1000 and Average rating is 6 and above
> > - Removed all duplicates from our joining point primary key Movie_ID and foreign Key Person_ID

In [83]:
#Select relevant data for computation and set certain cleaning papmeters such as region = US, Death_year Is null and avgrating >=6
final_q = pd.read_sql("""
SELECT mb.movie_id, mb.genres, mr.averagerating AS avg_ratings, mr.numvotes AS num_votes, mk.region, 
pr.category, p.primary_name AS artist_name, mb.runtime_minutes AS screen_time, mb.primary_title AS movie_title, p.death_year
FROM movie_basics mb
JOIN movie_akas mk ON mk.movie_id = mb.movie_id
JOIN movie_ratings mr ON mr.movie_id = mb.movie_id
JOIN principals pr ON pr.movie_id = mb.movie_id
JOIN persons p ON p.person_id = pr.person_id
WHERE mk.region != 'None'
AND p.death_year IS NULL
;""", conn)
#More cleaning using pandas, dropped duplicates, set the defining catagories and dropped missing/empty values from Genre
final_q = final_q.drop_duplicates(subset=['movie_id'])
# Removing duplicate movie names
final_q = final_q.drop_duplicates(subset=['movie_title'])
final_q = final_q.dropna(subset=['genres'])
categories_of_interest = ['actor', 'actress', 'writer', 'director', 'producer']
final_q = final_q[final_q['category'].isin(categories_of_interest)]
#Created a weighted average for rating
final_q['weighted_avg'] = (final_q['avg_ratings'] * final_q['num_votes']) / final_q['num_votes'].sum()
final_q

,movie_id,genres,avg_ratings,num_votes,region,category,artist_name,screen_time,movie_title,death_year,weighted_avg
0,tt0369610,"Action,Adventure,Sci-Fi",7.0,539338,BG,producer,Patrick Crowley,124.0,Jurassic World,None,1.666936e-02
423,tt0381957,Drama,7.6,23,GB,actor,Derek Jacobi,92.0,Cloud Cuckoo Land,None,7.717939e-07
433,tt0401729,"Action,Adventure,Sci-Fi",6.6,241792,UA,actor,Willem Dafoe,132.0,John Carter,None,7.046051e-03
750,tt10084780,"Drama,Romance",8.3,30,IN,director,Sateesh Chandra Nadella,NaN,Vinara sodara veera kumara,None,1.099409e-06
760,tt1014759,"Adventure,Family,Fantasy",6.5,358813,IE,actor,Johnny Depp,108.0,Alice in Wonderland,None,1.029773e-02
...,...,...,...,...,...,...,...,...,...,...,...
1992082,tt9593792,Horror,6.1,39,TH,director,Mate Yimsomboon,NaN,Ghost Wife,None,1.050399e-06
1992100,tt9644084,Documentary,5.2,5,DE,director,Uli M. Schüppel,95.0,Der Atem,None,1.147977e-07
1992120,tt9657946,Drama,6.0,16,XWW,actor,Marcelo Di Souza,90.0,Homing,None,4.238685e-07
1992144,tt9726638,"Action,Adventure,Comedy",6.0,6,CN,actress,Carman Lee,90.0,Monkey King: The Volcano,None,1.589507e-07


# CSV Files
> - Uploaded Zipped csv files and printed columns
> - Dropped Not needed columns from each file
> - Decide to also drop data with more that 50% of value missing

In [84]:
# Reading the movie info data
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t', compression='gzip')

# Dropping unnecessary columns from movie info data
movie_info_drop = ['box_office', 'currency', 'dvd_date', 'studio']
movie_info = movie_info.drop(columns=movie_info_drop)

# Printing the columns of movie info data
print("movie_info columns:")
print(list(movie_info.columns))
print()

# Reading the movie gross data
movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')

# Printing the columns of movie gross data
print("movie_gross columns:")
print(list(movie_gross.columns))
print()

# Reading the movie budget data
movie_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# Printing the columns of movie budget data
print("movie_budget columns:")
print(list(movie_budget.columns))
print()

# Reading the movie reviews data
movie_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', compression='gzip', encoding='latin-1')

# Dropping unnecessary column from movie reviews data
movie_review_drop = ['rating']
movie_reviews = movie_reviews.drop(columns=movie_review_drop)

# Printing the columns of movie reviews data
print("movie_reviews columns:")
print(list(movie_reviews.columns))
print()

# Reading the tmdb movie data
tmbd_movie = pd.read_csv('zippedData/tmdb.movies.csv.gz')

# Printing the columns of tmdb movie data
print("tmbd_movie columns:")
print(list(tmbd_movie.columns))
print()

movie_info columns:
['id', 'synopsis', 'rating', 'genre', 'director', 'writer', 'theater_date', 'runtime']

movie_gross columns:
['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']

movie_budget columns:
['id', 'release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']

movie_reviews columns:
['id', 'review', 'fresh', 'critic', 'top_critic', 'publisher', 'date']

tmbd_movie columns:
['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count']



# Movie Budget Cleaning
> - After analysing the CSV, we decided to use the Movie Budge data
> - We cleaned the data removing comma's and $ sign, assigning the column as integer
> - We removed duplicated from movies column to ease merging with SQL data
> - Calculated needed number to compute profit and budget better, and created columns for each such as ROI

In [85]:
# Converting release_date column to datetime format
movie_budget["release_date"] = pd.to_datetime(movie_budget["release_date"])

# Extracting release year and month from release_date
movie_budget["release_year"] = movie_budget["release_date"].dt.year
movie_budget["release_month"] = movie_budget["release_date"].dt.month

# Removing commas and dollar signs, and converting production_budget, domestic_gross, and worldwide_gross columns to integer format
movie_budget["production_budget"] = movie_budget["production_budget"].str.replace(",", "").str.replace("$", "").astype("int64")
movie_budget["domestic_gross"] = movie_budget["domestic_gross"].str.replace(",", "").str.replace("$", "").astype("int64")
movie_budget["worldwide_gross"] = movie_budget["worldwide_gross"].str.replace(",", "").str.replace("$", "").astype("int64")

movie_budget["production_budget"] = movie_budget["production_budget"] / 1000000
movie_budget["domestic_gross"] = movie_budget["domestic_gross"] / 1000000
movie_budget["worldwide_gross"] = movie_budget["worldwide_gross"] / 1000000

# Removing duplicate movie names 
movie_budget = movie_budget.drop_duplicates(subset=['movie'])

# Removing rows where worldwide_gross is 0
movie_budget = movie_budget[movie_budget['worldwide_gross'] != 0]

# Calculating foreign_gross as the difference between worldwide_gross and domestic_gross
movie_budget["foreign_gross"] = movie_budget["worldwide_gross"] - movie_budget["domestic_gross"]

# Calculating total_profit as the difference between worldwide_gross and production_budget
movie_budget["total_profit"] = movie_budget["worldwide_gross"] - movie_budget["production_budget"]

# Calculating ROI (Return on Investment) as the ratio of worldwide_gross to production_budget multiplied by 100
movie_budget['ROI'] = (movie_budget['total_profit'] / movie_budget['production_budget']) * 100

# Merge Movie_Budget with Merged_SQL

In [92]:
# merge movie_basics and movie_budget on movie and primary_title
final_data = pd.merge(final_q, movie_budget, left_on="movie_title", right_on="movie", how="inner")
final_data

,movie_id,genres,avg_ratings,num_votes,region,category,artist_name,screen_time,movie_title,death_year,...,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,foreign_gross,total_profit,ROI
0,tt0369610,"Action,Adventure,Sci-Fi",7.0,539338,BG,producer,Patrick Crowley,124.0,Jurassic World,None,...,2015-06-12,Jurassic World,215.0,652.270625,1648.854864,2015,6,996.584239,1433.854864,666.909239
1,tt0401729,"Action,Adventure,Sci-Fi",6.6,241792,UA,actor,Willem Dafoe,132.0,John Carter,None,...,2012-03-09,John Carter,275.0,73.058679,282.778100,2012,3,209.719421,7.778100,2.828400
2,tt1014759,"Adventure,Family,Fantasy",6.5,358813,IE,actor,Johnny Depp,108.0,Alice in Wonderland,None,...,2010-03-05,Alice in Wonderland,200.0,334.191110,1025.491110,2010,3,691.300000,825.491110,412.745555
3,tt1194173,"Action,Adventure,Thriller",6.7,268678,RS,actor,Scott Glenn,135.0,The Bourne Legacy,None,...,2012-08-10,The Bourne Legacy,125.0,113.203870,280.355920,2012,8,167.152050,155.355920,124.284736
4,tt1219289,"Mystery,Sci-Fi,Thriller",7.4,492490,AR,actor,Robert De Niro,105.0,Limitless,None,...,2011-03-18,Limitless,27.0,79.249455,155.564919,2011,3,76.315464,128.564919,476.166367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,tt3829266,"Action,Adventure,Sci-Fi",5.4,94888,LT,director,Shane Black,107.0,The Predator,None,...,2018-09-14,The Predator,88.0,51.024708,159.487563,2018,9,108.462855,71.487563,81.235867
1713,tt4651520,Comedy,6.2,100952,UA,actress,Christina Applegate,100.0,Bad Moms,None,...,2016-07-29,Bad Moms,20.0,113.257297,180.998716,2016,7,67.741419,160.998716,804.993580
1714,tt5462602,"Comedy,Drama,Romance",7.6,104008,NO,actress,Holly Hunter,120.0,The Big Sick,None,...,2017-06-23,The Big Sick,5.0,42.872467,53.972860,2017,6,11.100393,48.972860,979.457200
1715,tt5649108,"Comedy,Crime,Drama",6.7,25098,FR,producer,Nat Faxon,92.0,Thoroughbreds,None,...,2018-03-09,Thoroughbreds,5.0,3.072605,3.187302,2018,3,0.114697,-1.812698,-36.253960


> - Seperating and analysing the Genre column
> - We made the first listed genre the main genre for further analysis
> - We found the 5 most appearing Genre with highest ratings

In [93]:
#Extracts first genre in the list as main genre
def get_main_genre(x):
    genres = x.split(',')
    main_genre = genres[0]
    return main_genre
final_data['main_genre'] = final_data['genres'].apply(lambda x: get_main_genre(x))

In [94]:
#create subgenres column
def get_subgenres(x):
    genres = x.split(',')
    sub_genres = genres[1:]
    return sub_genres
final_data['subgenres'] = final_data['genres'].apply(lambda x: get_subgenres(x))

In [95]:
final_data

,movie_id,genres,avg_ratings,num_votes,region,category,artist_name,screen_time,movie_title,death_year,...,production_budget,domestic_gross,worldwide_gross,release_year,release_month,foreign_gross,total_profit,ROI,main_genre,subgenres
0,tt0369610,"Action,Adventure,Sci-Fi",7.0,539338,BG,producer,Patrick Crowley,124.0,Jurassic World,None,...,215.0,652.270625,1648.854864,2015,6,996.584239,1433.854864,666.909239,Action,"[Adventure, Sci-Fi]"
1,tt0401729,"Action,Adventure,Sci-Fi",6.6,241792,UA,actor,Willem Dafoe,132.0,John Carter,None,...,275.0,73.058679,282.778100,2012,3,209.719421,7.778100,2.828400,Action,"[Adventure, Sci-Fi]"
2,tt1014759,"Adventure,Family,Fantasy",6.5,358813,IE,actor,Johnny Depp,108.0,Alice in Wonderland,None,...,200.0,334.191110,1025.491110,2010,3,691.300000,825.491110,412.745555,Adventure,"[Family, Fantasy]"
3,tt1194173,"Action,Adventure,Thriller",6.7,268678,RS,actor,Scott Glenn,135.0,The Bourne Legacy,None,...,125.0,113.203870,280.355920,2012,8,167.152050,155.355920,124.284736,Action,"[Adventure, Thriller]"
4,tt1219289,"Mystery,Sci-Fi,Thriller",7.4,492490,AR,actor,Robert De Niro,105.0,Limitless,None,...,27.0,79.249455,155.564919,2011,3,76.315464,128.564919,476.166367,Mystery,"[Sci-Fi, Thriller]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,tt3829266,"Action,Adventure,Sci-Fi",5.4,94888,LT,director,Shane Black,107.0,The Predator,None,...,88.0,51.024708,159.487563,2018,9,108.462855,71.487563,81.235867,Action,"[Adventure, Sci-Fi]"
1713,tt4651520,Comedy,6.2,100952,UA,actress,Christina Applegate,100.0,Bad Moms,None,...,20.0,113.257297,180.998716,2016,7,67.741419,160.998716,804.993580,Comedy,[]
1714,tt5462602,"Comedy,Drama,Romance",7.6,104008,NO,actress,Holly Hunter,120.0,The Big Sick,None,...,5.0,42.872467,53.972860,2017,6,11.100393,48.972860,979.457200,Comedy,"[Drama, Romance]"
1715,tt5649108,"Comedy,Crime,Drama",6.7,25098,FR,producer,Nat Faxon,92.0,Thoroughbreds,None,...,5.0,3.072605,3.187302,2018,3,0.114697,-1.812698,-36.253960,Comedy,"[Crime, Drama]"


In [90]:
final_data = final_data[["movie", "genres", 'main_genre', 'subgenres', 'release_month', 'release_year',"production_budget", "domestic_gross", "foreign_gross", "worldwide_gross", "total_profit", "ROI"]]

In [96]:
final_data.to_csv('final_data.csv', index =False)

In [ ]:
#Sorts the final_data by the ROI column in descending order:
sorted_data = final_data.sort_values(by='ROI', ascending=False)

#The top 10 rows from the sorted DataFrame:
top_10_data = sorted_data.head(20)

popular_genre = top_10_data[['main_genre', 'subgenres', 'ROI']]
popular_genre

# Data Visualizations

In [ ]:
budget_stats = final_data['production_budget'].describe()
highest_budget = budget_stats['max']
lowest_budget = budget_stats['min']
median_budget = budget_stats['50%']

roi_stats = final_data['ROI'].describe()
highest_roi = roi_stats['max']
lowest_roi = roi_stats['min']
median_roi = roi_stats['50%']
print("High budget:", highest_budget)
print("Low budget:",lowest_budget)
print("Median budget:",median_budget)

print("High ROI:", highest_roi)
print("Low ROI:",lowest_roi)
print("Median ROI:",median_roi)

In [ ]:
# def budget(x): 
#     if x < 5000000: 
#         budget = 'low'
#     if 5000000 <= x < 50000000: 
#         budget = 'mid'
#     if x >= 50000000:
#         budget = 'high'
#     return budget

# final_data['budget'] = final_data['production_budget'].apply(lambda x: budget(x))